In [86]:
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict
import pandas as pd
import api


data = pd.read_csv('input/data.csv')

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=api.CLIENT_ID,
                                                           client_secret=api.CLIENT_SECRET))

In [87]:

relevant_features = ['valence', 'acousticness', 'danceability', 'energy',
               'instrumentalness', 'key', 'liveness', 'loudness',
               'mode', 'speechiness', 'tempo']
X = data.loc[:,relevant_features]

In [88]:
def find_song(name, artist):
    song_data = defaultdict()
    results = sp.search(q='track: {} artist: {}'.format(name, artist), limit=1)
    if not results['tracks']['items']:
        return None

    results = results['tracks']['items'][0]
    track_id = results['id']
    audio_features = sp.audio_features(track_id)[0]
    print('yo')
    song_data['name'] = [name]
    song_data['explicit'] = [int(results['explicit'])]
    song_data['duration_ms'] = [results['duration_ms']]
    song_data['popularity'] = [results['popularity']]

    for key, value in audio_features.items():
        song_data[key] = value

    return pd.DataFrame(song_data)


In [89]:
def get_song_data(song):


    song_data = data[data.name.str.contains(song['name'],case=False)
                     & data.artists.str.contains(song['artist'], case=False)]

    if len(song_data) == 0:
        song_data = find_song(song['name'], song['artist'])
    return song_data





In [90]:
data[relevant_features].head()

,valence,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo
0,0.0594,0.982,0.279,0.211,0.878000,10,0.665,-20.096,1,0.0366,80.954
1,0.9630,0.732,0.819,0.341,0.000000,7,0.160,-12.441,1,0.4150,60.936
2,0.0394,0.961,0.328,0.166,0.913000,3,0.101,-14.850,1,0.0339,110.339
3,0.1650,0.967,0.275,0.309,0.000028,5,0.381,-9.316,1,0.0354,100.109
4,0.2530,0.957,0.418,0.193,0.000002,3,0.229,-10.096,1,0.0380,101.665


In [91]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
#
# cols_to_transform = [0,1,2,3,4,5,6,7,8,9,10]
# transformer = ColumnTransformer(transformers=[('scaler', StandardScaler(), cols_to_transform)], remainder='passthrough')
#
#
# scaled_data = pd.DataFrame(transformer.fit_transform(data[relevant_features]), columns=transformer.get_feature_names_out())

In [92]:
# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(data[relevant_features])

In [93]:
song_name = input('Enter song name: ')
artist = input('Enter artist name: ')


In [94]:
song = {'name': song_name, 'artist': artist}

song_data = get_song_data(song)
song_data

yo


,name,explicit,duration_ms,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,time_signature
0,beautiful maria of my soul,0,252467,31,0.593,0.312,7,-10.639,0,0.0258,...,0.000025,0.434,0.506,91.508,audio_features,6ugaw1nIeza8vfIF0uvTzz,spotify:track:6ugaw1nIeza8vfIF0uvTzz,https://api.spotify.com/v1/tracks/6ugaw1nIeza8...,https://api.spotify.com/v1/audio-analysis/6uga...,4


In [95]:

song_data = song_data[relevant_features]
song_data

,valence,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo
0,0.506,0.623,0.593,0.312,0.000025,7,0.434,-10.639,0,0.0258,91.508


In [96]:
song_data = song_data.iloc[0,:].values.reshape(1,-1)

In [97]:
# scaled_song_data = pd.DataFrame(transformer.transform(song_data), columns=transformer.get_feature_names_out())
# scaled_song_data

In [98]:
# scaled_song_data = scaler.transform(song_data)
# scaled_song_data

In [99]:
from scipy.spatial.distance import cdist

distances = cdist(song_data, X, 'cosine')

In [100]:
index = list(np.argsort(distances)[:, :5][0])
index

[39779, 33079, 82513, 45489, 40456]

In [101]:

rec_songs = data.iloc[index]
rec_songs = rec_songs.loc[:,['name', 'artists', 'release_date', 'id']]

In [102]:
rec_songs.to_csv('recommendations1.csv', index=False)